In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.cluster import KMeans

In [2]:
%%time
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

Wall time: 59.5 s


特徴量作成コードを記載

In [3]:
test_oof = pd.read_csv("./oof/102_LightGBM_tests.csv")

In [14]:
pseudo_train=test_df
pseudo_train['oof_lgbm']=test_oof
pseudo_train.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,oof_lgbm
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197,0.551902
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760,0.712861
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794,0.607608
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108,0.604207
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846,0.264500


In [5]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [15]:
pseudo_train_1=pseudo_train.sort_values('oof_lgbm').head(3000)
pseudo_train_1['target']=1

In [16]:
pseudo_train_0=pseudo_train.sort_values('oof_lgbm',ascending=False).head(3000)
pseudo_train_0['target']=0
len(pseudo_train_0)

3000

In [17]:
pseudo_train=pd.concat([pseudo_train_1,pseudo_train_0])
pseudo_train

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,oof_lgbm,target
105706,test_105706,10.3936,-6.4189,9.5247,7.8099,11.1715,-17.7475,5.8850,13.4918,0.1781,...,-0.5474,8.0819,15.0911,-1.1233,2.5956,9.7043,14.8579,-1.2989,0.001122,1
39847,test_39847,9.9934,-2.5523,13.0469,4.9809,8.7022,-16.7211,4.8072,20.2088,-2.1886,...,1.6396,2.2581,19.2425,-1.3597,-3.5639,9.8502,14.6397,4.1005,0.001596,1
115231,test_115231,8.3262,-3.3312,10.2714,7.3695,8.5510,-1.9094,5.1248,17.4463,3.9706,...,3.1365,0.5838,14.9236,0.8383,-4.0744,8.5480,18.7958,-1.2773,0.001825,1
101683,test_101683,13.6989,-4.9092,10.0903,4.5934,12.4331,-11.6647,3.6202,13.7181,1.9564,...,5.0662,4.1376,17.4099,1.9204,-3.1459,8.7686,16.4888,-2.8526,0.001950,1
80579,test_80579,7.1220,-4.3181,8.4021,7.8886,11.2726,-6.1362,4.2388,18.3312,-1.3697,...,3.4527,4.4712,18.5695,1.6735,4.2979,9.8923,19.9980,-3.3327,0.002197,1
78551,test_78551,19.0349,-3.0647,11.4783,7.7379,8.9165,-16.1400,4.8162,14.9705,4.6115,...,1.8702,-1.4126,22.0249,0.2615,-3.4254,9.4055,22.2403,19.6290,0.002243,1
74893,test_74893,10.3650,-0.9690,11.2845,4.7028,11.7736,-10.6010,5.1009,20.5455,3.9762,...,3.8531,5.5408,22.5961,0.4503,-6.5491,9.2124,12.9239,-10.4070,0.002263,1
181965,test_181965,6.9163,-3.8266,10.6719,8.0066,12.6409,-9.5153,3.6845,15.5155,-4.8905,...,1.3153,1.1940,15.4783,0.8653,-2.8314,8.7435,19.5978,-19.3692,0.002334,1
77237,test_77237,13.1936,2.5410,8.4642,4.0345,8.5469,2.1452,4.9857,13.2056,-2.3320,...,1.9027,-0.5917,14.6925,-1.4617,4.0439,9.4350,20.3542,-19.0887,0.002359,1
113880,test_113880,10.3636,-3.5616,9.5031,6.9667,10.7778,-9.2855,3.2480,21.2061,4.9750,...,5.2394,12.4961,15.0731,-1.7499,8.7042,10.1133,18.3777,-0.5404,0.002439,1


In [18]:
pseudo_train=pseudo_train.drop(['oof_lgbm'],axis=1)

In [19]:
pseudo_train

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,target
105706,test_105706,10.3936,-6.4189,9.5247,7.8099,11.1715,-17.7475,5.8850,13.4918,0.1781,...,7.8516,-0.5474,8.0819,15.0911,-1.1233,2.5956,9.7043,14.8579,-1.2989,1
39847,test_39847,9.9934,-2.5523,13.0469,4.9809,8.7022,-16.7211,4.8072,20.2088,-2.1886,...,7.8804,1.6396,2.2581,19.2425,-1.3597,-3.5639,9.8502,14.6397,4.1005,1
115231,test_115231,8.3262,-3.3312,10.2714,7.3695,8.5510,-1.9094,5.1248,17.4463,3.9706,...,2.3797,3.1365,0.5838,14.9236,0.8383,-4.0744,8.5480,18.7958,-1.2773,1
101683,test_101683,13.6989,-4.9092,10.0903,4.5934,12.4331,-11.6647,3.6202,13.7181,1.9564,...,7.2323,5.0662,4.1376,17.4099,1.9204,-3.1459,8.7686,16.4888,-2.8526,1
80579,test_80579,7.1220,-4.3181,8.4021,7.8886,11.2726,-6.1362,4.2388,18.3312,-1.3697,...,3.8503,3.4527,4.4712,18.5695,1.6735,4.2979,9.8923,19.9980,-3.3327,1
78551,test_78551,19.0349,-3.0647,11.4783,7.7379,8.9165,-16.1400,4.8162,14.9705,4.6115,...,8.1792,1.8702,-1.4126,22.0249,0.2615,-3.4254,9.4055,22.2403,19.6290,1
74893,test_74893,10.3650,-0.9690,11.2845,4.7028,11.7736,-10.6010,5.1009,20.5455,3.9762,...,7.9883,3.8531,5.5408,22.5961,0.4503,-6.5491,9.2124,12.9239,-10.4070,1
181965,test_181965,6.9163,-3.8266,10.6719,8.0066,12.6409,-9.5153,3.6845,15.5155,-4.8905,...,6.0839,1.3153,1.1940,15.4783,0.8653,-2.8314,8.7435,19.5978,-19.3692,1
77237,test_77237,13.1936,2.5410,8.4642,4.0345,8.5469,2.1452,4.9857,13.2056,-2.3320,...,7.1983,1.9027,-0.5917,14.6925,-1.4617,4.0439,9.4350,20.3542,-19.0887,1
113880,test_113880,10.3636,-3.5616,9.5031,6.9667,10.7778,-9.2855,3.2480,21.2061,4.9750,...,5.0835,5.2394,12.4961,15.0731,-1.7499,8.7042,10.1133,18.3777,-0.5404,1


In [20]:
train_df

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
5,train_5,0,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
6,train_6,0,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
7,train_7,0,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
8,train_8,0,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,10.1854,...,1.4298,14.7510,1.6395,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453
9,train_9,0,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [21]:
pseudo_train=pseudo_train.ix[:,train_df.columns]
pseudo_train

C:\Users\tatsuya.okamoto\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
105706,test_105706,1,10.3936,-6.4189,9.5247,7.8099,11.1715,-17.7475,5.8850,13.4918,...,4.7598,7.8516,-0.5474,8.0819,15.0911,-1.1233,2.5956,9.7043,14.8579,-1.2989
39847,test_39847,1,9.9934,-2.5523,13.0469,4.9809,8.7022,-16.7211,4.8072,20.2088,...,-2.1867,7.8804,1.6396,2.2581,19.2425,-1.3597,-3.5639,9.8502,14.6397,4.1005
115231,test_115231,1,8.3262,-3.3312,10.2714,7.3695,8.5510,-1.9094,5.1248,17.4463,...,4.3945,2.3797,3.1365,0.5838,14.9236,0.8383,-4.0744,8.5480,18.7958,-1.2773
101683,test_101683,1,13.6989,-4.9092,10.0903,4.5934,12.4331,-11.6647,3.6202,13.7181,...,-2.4383,7.2323,5.0662,4.1376,17.4099,1.9204,-3.1459,8.7686,16.4888,-2.8526
80579,test_80579,1,7.1220,-4.3181,8.4021,7.8886,11.2726,-6.1362,4.2388,18.3312,...,1.3739,3.8503,3.4527,4.4712,18.5695,1.6735,4.2979,9.8923,19.9980,-3.3327
78551,test_78551,1,19.0349,-3.0647,11.4783,7.7379,8.9165,-16.1400,4.8162,14.9705,...,4.3980,8.1792,1.8702,-1.4126,22.0249,0.2615,-3.4254,9.4055,22.2403,19.6290
74893,test_74893,1,10.3650,-0.9690,11.2845,4.7028,11.7736,-10.6010,5.1009,20.5455,...,-6.4659,7.9883,3.8531,5.5408,22.5961,0.4503,-6.5491,9.2124,12.9239,-10.4070
181965,test_181965,1,6.9163,-3.8266,10.6719,8.0066,12.6409,-9.5153,3.6845,15.5155,...,7.6198,6.0839,1.3153,1.1940,15.4783,0.8653,-2.8314,8.7435,19.5978,-19.3692
77237,test_77237,1,13.1936,2.5410,8.4642,4.0345,8.5469,2.1452,4.9857,13.2056,...,1.9000,7.1983,1.9027,-0.5917,14.6925,-1.4617,4.0439,9.4350,20.3542,-19.0887
113880,test_113880,1,10.3636,-3.5616,9.5031,6.9667,10.7778,-9.2855,3.2480,21.2061,...,0.8410,5.0835,5.2394,12.4961,15.0731,-1.7499,8.7042,10.1133,18.3777,-0.5404


In [22]:
train_df=pd.concat([train_df,pseudo_train])

In [23]:
train_df.to_csv('./feature/103_train_pseudo.csv',index=False)